Copyright 2025 Quantinuum (www.quantinuum.com)

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

## Steane encoder demonstration

This notebook provides a high-level overview of the `h2xh2.encode.steane` functionalities.
With this package, users can play with the quantum error correction (QEC) code on top of `pytket` tools,
in particular the [[7, 1, 3]] code (Steane code).
One can encode a logical circuit in the fully fault-tolerant (FT) manner for the Clifford operations,
and partially FT for the non-Clifford operations such as R<sub>Z</sub> and T gates. 

Below are supported QEC components for Clifford operations:

- FT state preparation (Goto state preparation)
- FT measurement (error correction for the destructive measurement outcomes)
- Steane QEC for X or Z syndromes

This package assumes the Clifford+R<sub>Z</sub> gate set.
One can increase the FT level as:

- Iceberg-style R<sub>Z</sub> implemented as R<sub>ZZZ</sub> (non-FT).
- Gate teleportation protocols:
    - Non-FT
    - + FT-state preparation with repeat-until-success (RUS)
    - + FT-measurement
    - + Iceberg-style quantum error detection (QED) with RUS

One can easily mix and match these QEC components to control the level of FT.
Ideally, the more QEC components, the better fidelity, but it is not that simple mainly due to the incoherent memory error,
as the QEC setup cause longer ion idling and/or transportation time.

To make the most of the target hardware, one may want to tweak between the different QEC settings.
This `h2xh2.encode.steane` packaged is designed for facilitating such a step-by-step approach to find a sweet spot of performance.

In [1]:
# Import section.
from pytket.circuit import Circuit
from pytket.backends.backendresult import BackendResult
from pytket.circuit.display import render_circuit_jupyter
from h2xh2.encode import steane
from pytket.extensions.quantinuum import (
    QuantinuumBackend,
    QuantinuumAPIOffline,
)

In [2]:
# Prepare Quantinuum backend and some utility function.ProcessLookupError
backend = QuantinuumBackend(
    device_name="H2-1LE",
    api_handler=QuantinuumAPIOffline(),
)

# Backend settings for convenience.
def run_circuit(
    backend: QuantinuumBackend,
    circ: Circuit,
    n_shots: int,
) -> BackendResult:
    cmp_circ = backend.get_compiled_circuit(circ, optimisation_level=0)
    results = backend.run_circuit(cmp_circ, n_shots=n_shots)
    return results

 ## Bell-pair measurement as a simple Clifford circuit example
 
 ### Non-FT implementation

In [3]:
# Non-FT implementation.
# Create a logical circuit.
log_circ = Circuit(2).H(0).CX(0, 1).measure_all()
print("Logical circuit:")
render_circuit_jupyter(log_circ)

Logical circuit:


In [4]:

from h2xh2.encode import encode
# Encode the logical circuit into the physical circuit (non-FT).
enc_circ = encode(log_circ)
print("Encoded circuit:")
render_circuit_jupyter(enc_circ)

Encoded circuit:


In [5]:
from h2xh2.encode import interpret

# Run circuit and show counts from the encoded backend result.
enc_result = run_circuit(backend, enc_circ, n_shots=10)
print("Encoded results:")
print(enc_result.get_bitlist())
print(enc_result.get_counts())

# Interpret the encoded result back in the logical space.
log_result = interpret(enc_result)
print("Logical results:")
print(log_result.get_counts())

Encoded results:
[c[0], c[1], c[2], c[3], c[4], c[5], c[6], c[7], c[8], c[9], c[10], c[11], c[12], c[13]]
Counter({(0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1): 1, (0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1): 1, (0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0): 1, (0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0): 1, (0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0): 1, (0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1): 1, (1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0): 1, (1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1): 1, (1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0): 1, (1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1): 1})
Logical results:
Counter({(1, 1): 6, (0, 0): 4})


### FT state preparation (Goto state preparation) and measurement

In [6]:
from h2xh2.encode import EncodeOptions, interpret, InterpretOptions, ReadoutMode 

# Create a logical circuit (Bell pair measurement).
log_circ = Circuit(2).H(0).CX(0, 1).measure_all()

# Encode the logical circuit into the physical circuit with FT state preparation.
enc_circ = encode(
    log_circ,
    options=EncodeOptions(
        ft_prep=True,
        n_rus_prep=1,
    ),
)
print("Encoded circuit:")
render_circuit_jupyter(enc_circ)

# Run circuit and show counts from the encoded backend result.
enc_result = run_circuit(backend, enc_circ, n_shots=20)
print("Encoded results:")
print(enc_result.get_bitlist())
print(enc_result.get_counts())

# Interpret the encoded result back in the logical space.
log_result = interpret(
    enc_result,
    options=InterpretOptions(
        readout_mode=ReadoutMode.Correct,
    ))
print("Logical results:")
print(log_result.get_counts())

Encoded circuit:


Encoded results:
[c[0], c[1], c[2], c[3], c[4], c[5], c[6], c[7], c[8], c[9], c[10], c[11], c[12], c[13]]
Counter({(1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0): 2, (1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0): 2, (0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1): 1, (0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1): 1, (0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1): 1, (0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0): 1, (0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1): 1, (0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1): 1, (0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1): 1, (0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1): 1, (1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0): 1, (1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0): 1, (1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0): 1, (1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0): 1, (1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1): 1, (1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0): 1, (1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0): 1, (1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1): 1})
Logical results:
Counter({(0, 0): 10, 

## Mid-circuit QEC and QED

### Steane QEC gadgets

In [7]:
from h2xh2.encode import steane_z_correct
# Prepare the |0> state preparation and measurement.
log_circ = Circuit(1)
log_circ.add_custom_gate(steane_z_correct, [], [0])
log_circ.measure_all()
render_circuit_jupyter(log_circ)

In [8]:
# Encode the logical circuit into the physical circuit.
enc_circ = encode(
    log_circ,
    options=EncodeOptions(
        ft_prep=True,
        n_rus_prep=1,
        ft_prep_synd=True,
        n_rus_synd=1,
    ),
)
print("Encoded circuit:")
render_circuit_jupyter(enc_circ)

Encoded circuit:


### Iceberg-style QED gadgets

In [9]:
from h2xh2.encode import iceberg_w_0_detect

log_circ = Circuit(1)
log_circ.add_custom_gate(iceberg_w_0_detect, [], [0])
render_circuit_jupyter(log_circ)

In [10]:
# Encode the logical circuit into the physical circuit.
enc_circ = encode(
    log_circ,
    options=EncodeOptions(),
)
render_circuit_jupyter(enc_circ)

 ## Rz as a basic FT non-Clifford implementation

 ### Iceberg-style implementation

In [11]:
# Iceberg-style Rz(0.25) = T implementation.
log_circ = Circuit(1)
log_circ.add_barrier(log_circ.qubits)
log_circ.Rz(0.25, 0)
log_circ.add_barrier(log_circ.qubits)
log_circ.measure_all()
enc_circ = encode(
    log_circ,
)
print("Logical circuit:")
render_circuit_jupyter(log_circ)
print("Encoded circuit:")
render_circuit_jupyter(enc_circ)

Logical circuit:


Encoded circuit:


### Non-FT T gate teleportation

In [12]:
from h2xh2.encode import RzMode, RzOptionsBinFracNonFT

# Non-FT gate teleportation Rz implementation.
log_circ = Circuit(1).T(0).measure_all()
enc_circ = encode(
    log_circ,
    options=EncodeOptions(
        rz_mode=RzMode.BIN_FRAC_NON_FT,
        rz_options=RzOptionsBinFracNonFT(
            max_bits=5,
        )
    )
)
render_circuit_jupyter(enc_circ)

### Non-FT $R_Z(x/2^{k})$: Recursive gate teleportation

One can implement the Rz gate with rotation angle smaller than $\pi/4$ by invoking recursive gate teleportation technique.
The rotation angle is automatically rounded to the binary fraction specified by `max_bits`.
This recursive gate teleportation is available for all the Rz implementation in this section.


In [13]:
# FT-measurement gate teleportation Rz implementation.
angle = 0.1234567
log_circ = Circuit(1).Rz(0.25 / 2, 0).measure_all()
enc_circ = encode(
    log_circ,
    options=EncodeOptions(
        rz_mode=RzMode.BIN_FRAC_NON_FT,
        rz_options=RzOptionsBinFracNonFT(max_bits=4)
    )
)
render_circuit_jupyter(enc_circ)

### Gate teleportation with FT measurement

In [14]:
from h2xh2.encode import RzOptionsBinFracMeasFT

# FT-SPAM gate teleportation Rz implementation.
log_circ = Circuit(1).Rz(0.25, 0).measure_all()
enc_circ = encode(
    log_circ,
    options=EncodeOptions(
        rz_mode=RzMode.BIN_FRAC_MEAS_FT,
        rz_options=RzOptionsBinFracMeasFT(
            max_bits=3,
        ),
    )
)
print("Encoded circuit")
render_circuit_jupyter(enc_circ)

Encoded circuit


### Gate teleportation with FT measurement + Iceberg QED (partially FT)

In [15]:
from h2xh2.encode import RzOptionsBinFracPartFT

log_circ = Circuit(1).Rz(0.25, 0).measure_all()
enc_circ = encode(
    log_circ,
    options=EncodeOptions(
        rz_mode=RzMode.BIN_FRAC_PART_FT,
        rz_options=RzOptionsBinFracPartFT(
            max_bits=5,
            max_rus=2,
        ),
    )
)
render_circuit_jupyter(enc_circ)

### Gate teleportation with FT prep + FT measurement + Iceberg QED (partially FT)

In [16]:
log_circ = Circuit(1).Rz(0.25, 0).measure_all()
enc_circ = encode(
    log_circ,
    options=EncodeOptions(
        rz_mode=RzMode.BIN_FRAC_PART_FT,
        rz_options=RzOptionsBinFracPartFT(
            max_bits=5,
            max_rus=2,
        ),
    )
)
render_circuit_jupyter(enc_circ)